In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pydicom
import os
import numpy
from matplotlib import pyplot, cm
from fastai.vision import *
import fastai

from os import walk 

## Prepare Data

In [ ]:
# function to open dcm images
def open_dcm_image(fn:PathOrStr, div:bool=True, convert_mode:str='L', cls:type=Image,
        after_open:Callable=None)->Image:
    "Return `Image` object created from image in file `fn`."
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        dicom_file = pydicom.dcmread(str(fn))
        arr = dicom_file.pixel_array.copy() 
        arr = arr * int(dicom_file.RescaleSlope) + int(dicom_file.RescaleIntercept) 
        level = 40; window = 80
        arr = np.clip(arr, level - window // 2, level + window // 2)
        x = PIL.Image.fromarray(arr).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(x)

In [ ]:
fastai.vision.data.open_image = open_dcm_image    

In [ ]:
dirpath = "../input/rsna-intracranial-hemorrhage-detection/"
!ls ../input/rsna-intracranial-hemorrhage-detection

In [ ]:
df_train = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv')
df_train['fn'] = df_train.ID.apply(lambda x: '_'.join(x.split('_')[:2]) + '.dcm')
df_train['label'] = df_train.ID.apply(lambda x: x.split('_')[-1])
df_train.head()
# remove corrupted file
df_train = df_train[df_train.fn != 'ID_000039fa0.dcm']
df_train = df_train[df_train.fn != 'ID_6431af929.dcm']

In [ ]:
print(df_train.shape)
df_train.drop_duplicates(inplace = True)
print(df_train.shape)

In [ ]:
df_train.head(2)

In [ ]:
pivot = df_train.pivot(index='fn', columns='label', values='Label')
pivot.reset_index(inplace=True)
pivot['any'].value_counts()

In [ ]:
mask = pivot['any'] == 0
pivot['None'] = ""
pivot.loc[mask, 'None'] = 'None'

label_cols = ['any', 'None'] 
for col in label_cols:
    print(col, end= ", ")
    pivot[col] = pivot[col].replace({0:"", 1:col})
    
pivot['MultiLabel'] = pivot[label_cols].apply(lambda x: " ".join((' '.join(x)).split()), axis=1)

In [ ]:
pivot.tail(2)

# Create DataBunch for fastai vision 

In [ ]:
tfms = get_transforms(do_flip = False) # we can add more augumentaion tricks here 

In [ ]:
pivot.MultiLabel.unique()

In [ ]:
path = "../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/"
np.random.seed(42)
data_train = (ImageList
.from_df(path=path,df= pivot[['fn', 'MultiLabel']])
.split_by_rand_pct(valid_pct=0.2)   #20% validation 
.label_from_df(cols=1, label_delim = ' ')
.transform(size=(128,128))
.databunch()
.normalize(imagenet_stats))

## Train Model

In [ ]:
path = '/output/'
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

In [ ]:
learn = cnn_learner( data_train, models.resnet34, path = path, metrics = acc_02 ) 
#we can try other pretrained models later

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-1
learn.freeze()  
learn.fit_one_cycle(2, slice(lr)) #2 epochs to save time 

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1')    

## Evaluate Model

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn, ds_type=DatasetType.Valid)
# interp.plot_top_losses(9)
# interp.plot_confusion_matrix() 

In [ ]:
data_classes = learn.data.classes
learn.data = data_train
y_predict, targets = learn.get_preds(ds_type=DatasetType.Valid) # on validation set 

In [ ]:
y_val= torch.Tensor.numpy(targets)
y_pred= torch.Tensor.numpy(y_predict)

In [ ]:
y_val=np.argmax(y_val,1 ) 
y_pred=np.argmax(y_pred,1 )  

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
sns.heatmap(confusion_matrix(y_val, y_pred), cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
confusion_matrix(y_val, y_pred) 

In [ ]:
import pandas as pd
report = classification_report(y_val, y_pred,output_dict=True) 
df = pd.DataFrame(report).transpose()
df